In [4]:
import requests
import pandas as pd
import myql #https://github.com/josuebrunel/myql
from yahoo_oauth import OAuth1
import logging
pd.options.display.max_columns=None

oauth = OAuth1(None, None, from_file='credentials.json')
logging.disable(logging.CRITICAL)
logging.disable(logging.DEBUG)
logging.disable(logging.INFO)
yql = myql.MYQL(format='json', oauth=oauth, diagnostics =False)

TABLES  
- fantasysports.draftresults team_key str
- fantasysports.leagues.standings 
- fantasysports.leagues.settings game_code="nfl"

In [7]:
resp=yql.raw_query('select * from fantasysports.draftresults where team_key="238.l.627060.t.8"')
resp.json()

{'query': {'count': 1,
  'created': '2017-08-17T15:56:10Z',
  'lang': 'en-US',
  'results': {'team': {'auction_budget_spent': '260',
    'auction_budget_total': '260',
    'draft_results': {'count': '23',
     'draft_result': [{'cost': '21',
       'pick': '35',
       'player_key': '238.p.7946',
       'round': '4',
       'team_key': '238.l.627060.t.8'},
      {'cost': '20',
       'pick': '36',
       'player_key': '238.p.7498',
       'round': '4',
       'team_key': '238.l.627060.t.8'},
      {'cost': '22',
       'pick': '40',
       'player_key': '238.p.7552',
       'round': '5',
       'team_key': '238.l.627060.t.8'},
      {'cost': '22',
       'pick': '42',
       'player_key': '238.p.7049',
       'round': '5',
       'team_key': '238.l.627060.t.8'},
      {'cost': '22',
       'pick': '43',
       'player_key': '238.p.7707',
       'round': '5',
       'team_key': '238.l.627060.t.8'},
      {'cost': '18',
       'pick': '45',
       'player_key': '238.p.8180',
       'roun

In [10]:
resp=yql.raw_query('select * from fantasysports.leagues where league_key="223.l.431"')
resp.json()

{'query': {'count': 1,
  'created': '2017-08-17T15:56:56Z',
  'lang': 'en-US',
  'results': {'league': {'allow_add_to_dl_extra_pos': '0',
    'current_week': '16',
    'draft_status': 'postdraft',
    'edit_key': '17',
    'end_date': '2009-12-28',
    'end_week': '16',
    'game_code': 'pnfl',
    'is_cash_league': '0',
    'is_finished': '1',
    'is_pro_league': '0',
    'league_id': '431',
    'league_key': '223.l.431',
    'league_type': 'private',
    'league_update_timestamp': '1262595518',
    'name': 'Y! Friends and Family League',
    'num_teams': '14',
    'renew': None,
    'renewed': '242_334732',
    'scoring_type': 'head',
    'season': '2009',
    'short_invitation_url': None,
    'start_date': '2009-09-10',
    'start_week': '1',
    'url': 'https://football.fantasysports.yahoo.com/archive/pnfl/2009/431',
    'weekly_deadline': None}}}}

In [9]:
yql.raw_query("select * from fantasysports.games where league='nfl'").content

b'{"error":{"lang":"en-US","description":"Cannot find required keys in where clause;  got \'league\', expecting required keys: (game_key,user_id) OR (game_key,use_login) OR (game_key)"}}'

YQL kinda sucks ^^. They require certain id fields so you can't easily get a list of nfl public teams. I had to manually derive some of the id's below

In [12]:
#from https://developer.yahoo.com/fantasysports/guide/
#NFL Public fantasy teams
game_id_mapping={
    #"175":"2007",
    "199": "2008",
    #"222": "2009",
    #"242": "2010",
    "257": "2011",
    #"273": "2012",
    #f"314": "2013",#had to guess and check...
    "331": "2014",
    #'348': "2015",
    '359': "2016"
    #"371": "2017"
   
}

In [13]:
def flattenDict(d):
    for key, val in d.items():
        if type(val)==dict:
            del d[key]
            for k,v in val.items():
                val[key+"_"+k]=v
                del val[k]
                d.update(val)
                return flattenDict(d)
    return d
league_info_rows=[]
league_standings_rows=[]
def flattenTeamStandings(json):
    league=json['query']['results']['league']
    
    league_info_row={}
    for key in league:
        if type(league[key])!=dict:
            league_info_row[key]=league[key]
        else:
            if key=="standings":
                for team in league['standings']['teams']['team']:
                    league_standings=flattenDict(team)
                    league_standings['league_id']=league['league_id']
                    league_standings_rows.append(flattenDict(team))
            else:
                print("nested", key)
    league_info_rows.append(league_info_row)
    
flattenTeamStandings(yql.raw_query("select * from fantasysports.leagues.standings where league_key='175.l.219648'").json())
pd.DataFrame(league_standings_rows)

,clinched_playoffs,coverage_value,draft_position,guid,has_draft_grade,league_id,league_scoring_type,losses,managers_manager_manager_id,name,nickname,number_of_moves,number_of_trades,outcome_totals_wins,percentage,playoff_seed,points_against,points_for,roster_adds_coverage_type,season,streak_type,team_id,team_key,team_logos_team_logo_size,team_points_coverage_type,team_standings_rank,ties,total,url,value,waiver_priority
0,1,17,3,--,0,219648,head,3,3,Hillbilly Phillies,--hidden--,9,0,11,.786,1,1172,1348.00,week,2007,win,3,175.l.219648.t.3,large,season,1,0,1348.00,https://s.yimg.com/dh/ap/fantasy/nfl/img/icon_...,4,5
1,1,17,8,NaN,0,219648,head,3,9,bears,--hidden--,15,0,11,.786,2,1027,1225.00,week,2007,win,9,175.l.219648.t.9,large,season,2,0,1225.00,https://s.yimg.com/dh/ap/fantasy/nfl/img/icon_...,8,1
2,1,17,5,NaN,0,219648,head,6,7,88 Green Wave,--hidden--,13,0,8,.571,4,1165,1332.00,week,2007,loss,7,175.l.219648.t.7,large,season,3,0,1332.00,https://s.yimg.com/dh/ap/fantasy/nfl/img/icon_...,1,8
3,1,17,7,--,0,219648,head,4,5,bombs away,--hidden--,10,0,9,.679,3,1152,1293.00,week,2007,tie,5,175.l.219648.t.5,large,season,4,1,1293.00,https://s.yimg.com/dh/ap/fantasy/nfl/img/icon_...,1,2
4,NaN,17,1,NaN,0,219648,head,6,4,The Beasts,--hidden--,8,1,8,.571,5,1033,1188.00,week,2007,loss,4,175.l.219648.t.4,large,season,5,0,1188.00,https://s.yimg.com/dh/ap/fantasy/img/profile_9...,1,7
5,NaN,17,2,NaN,0,219648,head,8,8,General Chukaki,--hidden--,10,1,6,.429,6,1239,1270.00,week,2007,win,8,175.l.219648.t.8,large,season,6,0,1270.00,https://s.yimg.com/dh/ap/fantasy/img/profile_9...,1,6
6,NaN,17,9,NaN,0,219648,head,10,6,Team to win,--hidden--,7,0,4,.286,8,1121,928.00,week,2007,loss,6,175.l.219648.t.6,large,season,7,0,928.00,https://s.yimg.com/dh/ap/fantasy/nfl/img/icon_...,3,9
7,NaN,17,6,NaN,0,219648,head,8,2,GENIE,--hidden--,8,0,5,.393,7,1205,1022.00,week,2007,tie,2,175.l.219648.t.2,large,season,8,1,1022.00,https://s.yimg.com/dh/ap/fantasy/img/profile_9...,1,3
8,NaN,17,4,NaN,0,219648,head,10,1,Cyber X,--hidden--,0,0,4,.286,NaN,1094,854.00,week,2007,win,1,175.l.219648.t.1,large,season,9,0,854.00,https://s.yimg.com/dh/ap/fantasy/nfl/img/icon_...,1,4
9,NaN,17,10,--,0,219648,head,11,10,ShLaGaBoYz77,--hidden--,3,0,3,.214,NaN,1226,974.00,week,2007,loss,10,175.l.219648.t.10,large,season,10,0,974.00,https://s.yimg.com/dh/ap/fantasy/nfl/img/icon_...,2,10


In [14]:
from time import sleep
import datetime
#~11 hours per "game_id" which is really a nfl fantasy season
#FYI this is kinda crap and should be improved.
ids_to_try=range(200000,300000)
N=len(ids_to_try)
league_info_rows=[]
league_standings_rows=[]
BASE_QUERY="select * from fantasysports.leagues.standings where league_key='GAME_ID.l.LEAGUE_ID'"
bad_ids=[]
for game_id in game_id_mapping.keys():
    print(game_id, game_id_mapping[game_id])
    for idx,league_id in enumerate(ids_to_try):
        query=BASE_QUERY.replace("GAME_ID",str(game_id)).replace("LEAGUE_ID",str(league_id))
        try:
            flattenTeamStandings(yql.raw_query(query).json())
        except:
            bad_ids.append(league_id)
            if len(bad_ids)%1000==0:
                print(len(bad_ids))
        if idx%1000==5:
            print(idx,N,idx/N, datetime.datetime.now())
            oauth = OAuth1(None, None, from_file='credentials.json')
            logging.disable(logging.CRITICAL)
            logging.disable(logging.DEBUG)
            logging.disable(logging.INFO)
            yql = myql.MYQL(format='json', oauth=oauth, diagnostics =False)
            pd.DataFrame(league_info_rows).to_csv("leage_info_rows-in-progressv5.csv",index=False)
            pd.DataFrame(league_standings_rows).to_csv("league_standings_rows-in-progressv5.csv",index=False)

199 2008
5 100000 5e-05 2017-08-17 10:59:44.528292


### Additional Info  
- [API Call to get Team Settings]( https://developer.yahoo.com/yql/console/#h=select+*+from+fantasysports.leagues.settings+where+league_key+in+('242.l.200007'%2C'%09242.l.200015')
- You can probably multi-thread the above or come up with a simpler way to check multiple keys at once. 